10.1.1

In [1]:
import requests
import json

url = "https://api.weather.gov/gridpoints/OKX/34,36/forecast"
response = requests.get(url)
print(response)

<Response [200]>


In [2]:
forecast = response.json()
print(forecast.keys())

dict_keys(['@context', 'type', 'geometry', 'properties'])


In [3]:
print(forecast["properties"]["periods"])

[{'number': 1, 'name': 'Memorial Day', 'startTime': '2025-05-26T13:00:00-04:00', 'endTime': '2025-05-26T18:00:00-04:00', 'isDaytime': True, 'temperature': 73, 'temperatureUnit': 'F', 'temperatureTrend': '', 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 2}, 'windSpeed': '5 mph', 'windDirection': 'NE', 'icon': 'https://api.weather.gov/icons/land/day/sct?size=medium', 'shortForecast': 'Mostly Sunny', 'detailedForecast': 'Mostly sunny. High near 73, with temperatures falling to around 71 in the afternoon. Northeast wind around 5 mph.'}, {'number': 2, 'name': 'Tonight', 'startTime': '2025-05-26T18:00:00-04:00', 'endTime': '2025-05-27T06:00:00-04:00', 'isDaytime': False, 'temperature': 56, 'temperatureUnit': 'F', 'temperatureTrend': '', 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 2}, 'windSpeed': '3 mph', 'windDirection': 'SW', 'icon': 'https://api.weather.gov/icons/land/night/sct?size=medium', 'shortForecast': 'Partly Cloudy', 'detailedFor

In [4]:
import pandas as pd

forecast_df = pd.DataFrame(forecast["properties"]["periods"])
forecast_df[
    ["endTime", "isDaytime", "temperature", "windSpeed"]
].head()

,endTime,isDaytime,temperature,windSpeed
0,2025-05-26T18:00:00-04:00,True,73,5 mph
1,2025-05-27T06:00:00-04:00,False,56,3 mph
2,2025-05-27T18:00:00-04:00,True,74,2 to 9 mph
3,2025-05-28T06:00:00-04:00,False,57,3 to 9 mph
4,2025-05-28T18:00:00-04:00,True,65,3 to 10 mph


In [5]:
import re

def get_wind_speed(row):
    numbers = [int(num) for num in re.findall(r'\d+', row)]
    return max(numbers)

forecast_df["date"] = pd.to_datetime(forecast_df['endTime']).dt.date

forecast_df["wind_speed"] = forecast_df["windSpeed"].apply(
    get_wind_speed
)  # D

forecast_df[["date", "temperature", "wind_speed"]].head() 

,date,temperature,wind_speed
0,2025-05-26,73,5
1,2025-05-27,56,3
2,2025-05-27,74,9
3,2025-05-28,57,9
4,2025-05-28,65,10


In [6]:
daily_forecast = (
    forecast_df.groupby("date")
    .agg({
        "temperature": ["min", "max"],
        "wind_speed": "max"})
    .reset_index()
)

daily_forecast.columns = [
"date", "low_temp", "high_temp", "wind_speed"
]

daily_forecast.head() 

,date,low_temp,high_temp,wind_speed
0,2025-05-26,73,73,5
1,2025-05-27,56,74,9
2,2025-05-28,57,65,10
3,2025-05-29,58,72,10
4,2025-05-30,61,76,13
